Read tweets into crate
=========
This requires an HTTP interface to crate running at http://localhost:8098/admin

In [1]:
import json
import configparser
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream
import itertools

In [2]:
#Load authentication 
#Secret file should be something like:
#[twitter]
#ACCESS_TOKEN=....
#ACCESS_SECRET=....
config = configparser.ConfigParser()
with open('twitter_secrets.ini', 'r') as configfile:
    config.read_file(configfile)


oauth = OAuth(config['twitter']['ACCESS_TOKEN'],
              config['twitter']['ACCESS_SECRET'],
              config['twitter']['CONSUMER_KEY'],
              config['twitter']['CONSUMER_SECRET'])

In [3]:
import riak
client = riak.RiakClient(pb_port=8087)
twitterBucket = client.bucket("tweets")

In [4]:
# Initiate the connection to Twitter Streaming API
twitterStream = TwitterStream(auth=oauth)

samples = twitterStream.statuses.sample()

for tweet in itertools.islice(samples, 1000):
    if ('user' in tweet) and ('id_str' in tweet) and ('text' in tweet):
        key = twitterBucket.new(tweet['id_str'], data = {
            'user.id_str': tweet['user']['id_str'],
            'user.name': tweet['user']['name'],
            'user.lang': tweet['user']['lang'],
            'user.followers_count': tweet['user']['followers_count'],
            'user.friends_count': tweet['user']['friends_count'],
            'user.statuses_count': tweet['user']['statuses_count'],
            'text': tweet['text'],
            'source': tweet['source'],
            'created_at': tweet['created_at']     
        })
        key.store()


In [5]:
tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Mar 10 14:05:51 +0000 2017',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': '7asnat.com',
    'expanded_url': 'http://7asnat.com/',
    'indices': [46, 69],
    'url': 'https://t.co/G0iN1MsnKQ'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 840202041981038592,
 'id_str': '840202041981038592',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'ar',
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://7asnat.com" rel="nofollow"> حسنات</a>',
 'text': 'اللهم ارزقنا العفو و العافية بالدنيا و الآخرة https://t.co/G0iN1MsnKQ',
 'timestamp_ms': '1489154751664',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'c